In [7]:
!pip install pandas
!pip install pyarrow
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 KB 838.2 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [54]:
import pandas as pd
from datetime import datetime, timedelta

# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file_path = 'example.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path, dtype=str)

prev = {'Position ID': '', 'Position Status': '', 'Time': '', 'Time Out': '', 'Timecard Hours (as Time)': '', 'Pay Cycle Start Date': '', 'Pay Cycle End Date': '', 'Employee Name': '', 'File Number': ''}
# checking for less than 10 hours but more than 1 hour between working shifts
store_distinct = set()
time_format = "%H:%M:%S"
time_break = timedelta()
lower_bound = timedelta(hours=1)
upper_bound = timedelta(hours=10)

for index, row in df.iterrows():
    curr = row.to_dict()
    if prev['Position ID'] == curr['Position ID']:
        if prev['Time'][0:10] == curr['Time'][0:10]:
            exit_time = prev['Time Out'][11:]
            entry_time = curr['Time'][11:]
            
            time1 = datetime.strptime(entry_time, time_format)
            time2 = datetime.strptime(exit_time, time_format)

            diff = time1-time2
            time_break += diff
            
        else:
            if lower_bound <= time_break <= upper_bound:
                store_distinct.add(curr['Position ID'])
            time_break = timedelta()
            prev = curr
    else:
        prev = curr
print(store_distinct)

# checking for more than 14 hours of work in a shift
prev_id = ''
time_sum = timedelta()
time_limit = timedelta(hours=14)
store_result = set()

for index, row in df.iterrows():
    data = row.to_dict()

    if data['Position ID'] == prev_id:
        # Convert the time string to a timedelta object
        curr_time = datetime.strptime(data['Timecard Hours (as Time)'] + ":00", time_format).time()
        curr_timedelta = timedelta(hours=curr_time.hour, minutes=curr_time.minute, seconds=curr_time.second)

        # Accumulate the time durations
        time_sum += curr_timedelta
    else:
        # Check if the accumulated time exceeds the limit
        if time_sum >= time_limit:
            store_result.add(prev_id)

        # Reset the time_sum for a new 'Position ID'
        time_sum = timedelta()
        prev_id = data['Position ID']

# Check for the last 'Position ID' after the loop
if time_sum >= time_limit:
    store_result.add(prev_id)

print(store_result)

# checking for consecutive work for 7 days

prev_id = ''
prev_date = '0000/00/0000'
date_format = "%y/%m/%d"
store_result2 = set()
count = 0

for index, row in df.iterrows():
    data = row.to_dict()

    if data['Position ID'] == prev_id:
        curr_date = data['Time'][0:10]
        # print(curr_date)
        date1 = datetime.strptime(curr_date, date_format)
        date2 = datetime.strptime(prev_date, date_format)

        if abs(date1 - date2) <= timedelta(days=2) and date1 != date2:
            count += 1
        else:
            count = 1  # Reset count to 1 for the current date match
    else:
        count = 1  # Reset count to 1 for a new 'Position ID'
        prev_id = data['Position ID']

    # Check if the count exceeds the threshold
    if count >= 7:
        store_result2.add(data['Position ID'])

    # Update previous date
    prev_date = str(data['Time'])[0:10]

# Check for the last 'Position ID' after the loop
if count >= 7:
    store_result2.add(prev_id)

print(store_result2)

        

{'WFS000528', 'WFS000412', 'WFS000579', 'WFS000503', 'WFS000228', 'WFS000523', 'WFS000511', 'WFS000571', 'WFS000318', 'WFS000226', 'WFS000420', 'WFS000462', 'WFS000491', 'WFS000521', 'WFS000588', 'WFS000268', 'WFS000577', 'WFS000551', 'WFS000573', 'WFS000522', 'WFS000183', 'WFS000372', 'WFS000180', 'WFS000557', 'WFS000457', 'WFS000550', 'WFS000368', 'WFS000517', 'WFS000473', 'WFS000555', 'WFS000572', 'WFS000589', 'WFS000556', 'WFS000515', 'WFS000200', 'WFS000585', 'WFS000360', 'WFS000495', 'WFS000153', 'WFS000442', 'WFS000509', 'WFS000127', 'WFS000536', 'WFS000391', 'WFS000456', 'WFS000584', 'WFS000246'}
{'WFS000345', 'WFS000508', 'WFS000412', 'WFS000579', 'WFS000446', 'WFS000503', 'WFS000546', 'WFS000507', 'WFS000477', 'WFS000571', 'WFS000226', 'WFS000462', 'WFS000538', 'WFS000588', 'WFS000101', 'WFS000501', 'WFS000551', 'WFS000306', 'WFS000573', 'WFS000183', 'WFS000550', 'WFS000500', 'WFS000556', 'WFS000552', 'WFS000569', 'WFS000509', 'WFS000536', 'WFS000584', 'WFS000393', 'WFS000587

ValueError: time data '2023-09-12' does not match format '%y/%m/%d'